# Job Description Processing

In [3]:
pip install gensim
pip install spacy

SyntaxError: invalid syntax (2348449629.py, line 1)

In [7]:
# Import Module
import os
import hashlib
import re
import string

import nltk
from gensim.models import Word2Vec
from nltk.tokenize import RegexpTokenizer, word_tokenize
import docx  # pip install python-docx
import fitz  # pip install PyMuPDF
from nltk.corpus import stopwords
import pandas as pd
import gensim
from gensim.models.phrases import Phraser, Phrases

from io import StringIO
from collections import Counter
nltk.download('omw-1.4')
from nltk.stem.wordnet import WordNetLemmatizer
from spacy.matcher import PhraseMatcher

ModuleNotFoundError: No module named 'gensim'

In [ ]:
# Folder path
path = "C:/Users/PSNR/Documents/Resume_project/Job_Descript"

os.chdir("C:/Users/PSNR/Documents/Resume_project")

In [ ]:
# Function to convert tokens to bigram words
def get_bigrams(sentence):
    tokens = nltk.word_tokenize(sentence)
    return zip(tokens, tokens[1:])

In [ ]:
# Function to find the bigram tokens from text
def filter_bigram_tokens(bigram_list, bigram_tokens):
    token=[]
    for i in range(len(bigram_tokens)):
        if bigram_tokens[i] in bigram_list:
            token.append(bigram_tokens[i])
            i+=2
        else:
            word = bigram_tokens[i][:bigram_tokens[i].find(" ")]
            token.append(word)
    return token

In [4]:
# Building Model using job description Text
def built_model(bigram_list):
    os.chdir(path)
    filename = []
    file_text = []
    frequent_word_list = []
    model_name=[]
    model_keywords_list=[]

    print("\n\n Processing the Job Description\n++++++++++++++++++++++++++++++++++++++++++\n")
    # iterate through all file
    for file in os.listdir():
        # Check whether file is in text format or not
        file_path = f"{path}\{file}"
        text = []
        if file.endswith(".docx"):

            doc = docx.Document(file_path)
            # Reading the document content
            for para in doc.paragraphs:
                text.append(para.text)

        if file.endswith(".pdf"):
            with fitz.open(file_path) as doc:
                for page in doc:
                    text.append(page.get_text())
        print(f"\n\nfile: {file}")


        x=[]
        for line in text:


            tokens = word_tokenize(line)
            tok = [w.lower() for w in tokens]
            table = str.maketrans('', '', string.punctuation)
            strpp = [w.translate(table) for w in tok]
            words = [word for word in strpp if word.isalpha()]
            stop_words = set(stopwords.words('english'))
            words = [w for w in words if not w in stop_words]
            token_text = " ".join(words)
            bigram_tokens = [' '.join(b) for b in get_bigrams(token_text.lower())]
            f_tokens = filter_bigram_tokens(bigram_list, bigram_tokens)
            lemmatizer = WordNetLemmatizer()
            lem_sent = [lemmatizer.lemmatize(words_sent) for words_sent in f_tokens]
            x.append(lem_sent)


        ### Building model

        # finding unique values
        texts = x
        ntexts =[]
        for line in texts:
            uniq_words = set(line)
            unique_list = []
            for w in uniq_words:
                if w not in unique_list:
                    unique_list.append(w)
            ntexts.append(unique_list)
            # Creating bigrams
        common_term = ["of", "with", "without", "and", "or", "the", "a","within"]

        x = ntexts
            # Create the relevant phrases from the list of sentences:
        phrases = Phrases(x, connector_words=common_term)
            # The Phraser object is used from now on to transform sentences
        bigram = Phraser(phrases)
            # Applying the Phraser to transform our sentences
        sentences = list(bigram[x])
            # Building word2vector model
        model = gensim.models.Word2Vec(sentences, min_count=2, workers=4, window=4, vector_size=500)
            # Saving the model
        os.chdir("C:/Users/PSNR/Documents/Resume_project/Models")
        model.save(f"{file}final.model")
        wrds = list(model.wv.index_to_key)
        print(f"model Tagwords: {wrds}")

        # 15 frequent words in the text     if it is not needed then, comment or delete from 115 to 128
        # Tokenizing the text
        tokenizer = RegexpTokenizer(r'\w+')
        tokens = tokenizer.tokenize(" ".join(text))
        tokens = [token for token in tokens if token not in stopwords.words('english')]
        token_text = " ".join(tokens)
        bigram_tokens = [' '.join(b) for b in get_bigrams(token_text.lower())]
        f_tokens = filter_bigram_tokens(bigram_list, bigram_tokens)
        print("\nFrequent words:")
        freq = Counter(f_tokens)
        max_list = [i for i in sorted(freq.values(),reverse=True)]
        for key, value in freq.items():
            if value in max_list[:10]:
                print(key, ":", value)

        filename.append(file)
        file_text.append(text)
        frequent_word_list.append(freq)
        model_name.append(f"{file}final.model")
        model_keywords_list.append(wrds)

    # Change the directory
    os.chdir("C:/Users/PSNR/Documents/Resume_project")
    # dictionary of lists
    job_dict = {'File name': filename, 'Text': file_text,'frequent_word_list':frequent_word_list,'Model_name':model_name,'Model_keywords_list':model_keywords_list}
    df = pd.DataFrame(job_dict)
    # saving the dataframe

In [5]:
os.chdir("C:/Users/PSNR/Documents/Resume_project")
bigram_list = []
# retreiving the bigram words
f = open('bigram_words.txt', 'r')
word_list = f.readlines()
for sub in word_list:
    bigram_list.append(re.sub('\n', '', sub))

NameError: name 'os' is not defined

In [ ]:
built_model(bigram_list)